In [102]:
import os
import time
import json
import math
import requests
import logging
import pandas as pd
import numpy as np

In [103]:
# Urls for fetching Data
url_oc      = "https://www.nseindia.com/option-chain"
url_bnf     = 'https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY'
url_nf      = 'https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'
url_indices = "https://www.nseindia.com/api/allIndices"

In [110]:
# Headers
headers = {'user-agent': 'Chrome/137.0.0.0',
            'accept-language': 'en-GB,en;q=0.9'}

sess = requests.Session()
cookies = dict()


In [111]:
cookies = dict()

# Local methods
def set_cookie():
    request = sess.get(url_nf, headers=headers, timeout=5)
    cookies = dict(request.cookies)

In [112]:
def get_data(url):
    set_cookie()
    response = sess.get(url, headers=headers, timeout=5, cookies=cookies)
    if(response.status_code==401):
        set_cookie()
        response = sess.get(url_nf, headers=headers, timeout=5, cookies=cookies)
    if(response.status_code==200):
        return response.text
    return ""

In [113]:
response = sess.get(url_bnf, headers=headers, timeout=20, cookies=cookies)

In [115]:
response.text

'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="UTF-8">\n  <title>Resource not found</title>\n</head>\n<body style="background-color: #e8e8e8;">\n  <div style="font-size: 32px; padding: 50px 25px; text-align: center;">\n    Resource not found\n  </div>\n</body>\n</html>\n'

In [84]:
def custom_round_to_nearest(x, num = 50):
    if x % 50 <= 25:
        return x - (x % num)
    else:
        return x + (num - (x % num))
    
def nearest_strike_bnf(x): return int(custom_round_to_nearest(x,100))
def nearest_strike_nf(x): return int(custom_round_to_nearest(x,50))



In [63]:
def get_data(url):
    set_cookie()
    response = sess.get(url, headers=headers, timeout=5, cookies=cookies)
    if(response.status_code==401):
        set_cookie()
        response = sess.get(url_nf, headers=headers, timeout=5, cookies=cookies)
    if(response.status_code==200):
        return response.text
    return ""

In [66]:
request = sess.get(url_nf, headers=headers, timeout=5)

In [67]:
request.text

'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="UTF-8">\n  <title>Resource not found</title>\n</head>\n<body style="background-color: #e8e8e8;">\n  <div style="font-size: 32px; padding: 50px 25px; text-align: center;">\n    Resource not found\n  </div>\n</body>\n</html>\n'

In [12]:
def set_header():
    global bnf_ul
    global nf_ul
    global bnf_nearest
    global nf_nearest
    response_text = get_data(url_indices)
    data = json.loads(response_text)
    for index in data["data"]:
        if index["index"]=="NIFTY 50":
            nf_ul = index["last"]
            logging.debug(f"NIFTY LTP is : {nf_ul}")
        if index["index"]=="NIFTY BANK":
            bnf_ul = index["last"]
            logging.debug(f"NIFTY BANK LTP is : {bnf_ul}")
    logging.info('Fetched latest NIFTY and BANK NIFTY LTP')
    logging.info(f'LTP NIFTY - {nf_ul} and BANK NIFTY - {bnf_ul}')
    bnf_nearest=nearest_strike_bnf(bnf_ul)
    nf_nearest=nearest_strike_nf(nf_ul)

In [13]:
def fetch_options(num,step,nearest,url):
    """
    num : Num of OTM's for CALL and PUT
    step: window of difference between each ATM
    nearest : rounded LTP Price
    url: url of the Scrip
    """
    strike = nearest - (step*num)
    start_strike = nearest - (step*num)
    response_text = get_data(url)
    data = json.loads(response_text)
    currExpiryDate = data["records"]["expiryDates"][0]
    sam_list = []
    for item in data['records']['data']:
        if item["expiryDate"] == currExpiryDate:
            if item["strikePrice"] == strike and item["strikePrice"] <= start_strike+(step*num*2):
                flattened_entry = {
                    'expiryDate': item['expiryDate'],
                    'CE_strikePrice': item['CE']['strikePrice'],
                    'CE expiryDate': item['CE']['expiryDate'],
                    'PE_strikePrice': item['PE']['strikePrice'],
                    'PE expiryDate': item['PE']['expiryDate'],
                    'CE_IV': item['CE']['impliedVolatility'],
                    'CE_LTP' : item['CE']['lastPrice'],
                    'strikePrice': item['strikePrice'],
                    'PE_LTP' : item['PE']['lastPrice'],# Flatten the 'CE' sub-dictionary
                    'PE_IV' : item['PE']['impliedVolatility'],  # Flatten the 'PE' sub-dictionary
                }
                strike = strike + step
                sam_list.append(flattened_entry)
    return sam_list

In [14]:
set_header()

data = fetch_options(4,50,nf_nearest,url_nf)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df = pd.DataFrame(data)
df= df[['expiryDate','CE_IV', 'CE_LTP', 'strikePrice', 'PE_LTP', 'PE_IV']]

In [20]:
response_text = get_data(url_indices)

In [21]:
response_text

'����b>\x00��an��\\\x16�ԏ��_�\'C�#iJ\x03+-�G\x0eY\x7f\uebf5>�\x05\x00ԗ!�>`��&��=��=�Ū�����#V����ޝO\x04*�Fŉh�r�<y�L�j> \x0b7�\x0f{G\x03!迾���.f�Q\x02*kޯ��w�(˿4�O�\x1d��\x7f��\x0f���t\\M�|k��L�������\x07���LǱ\x1e��Gc;\x0e������e\x1c������J�pl�ʕ���\x0f�\x13�!��J\x16�Z1\x08t��2u��&Y���&_�+#��o~$�z|�������Y\\p/]x\x127�z��C��槆d���в\x0b\x1bR8��o�k�ݿ�o�g��c;D��z|y�\x0b�R�X���S\x04�����Ň�����v��x��:%�z���[/�v�ؗ���%\r\x7f9�E�5\x01�����c������7�낱\x1e/�<��c�\x03w��x�\\��\n\x7f"�\x05o\x7f��ӧ��+\x08=�A�~�/\x02l�\x1c��^��dq�s��[����/�Vhd\x1c��8�s��������~�\x1f\x1b�oN�Η�������������N\x16g4ՂC�ϟ~���l*E�����o��HGR��k^\x1f�aztbx����KGr;u�I���z�\x1dQ����#�E��\x11a%H&"\x11�\x03��٨�\x07�@K\x1a�y{�\x11�\x19�[3��h�%`�FD9/A(�5ۿe�\x08��\x1ay\x1fٟ,\x00&�%\x1c��du0~��\ue988D��h����;�y����k\x17��1j\x1e\'�\x7f�Zz�[{����\x19fQ��V�bn�&�\x11\x0f�����y[p&�\x030r+\x10���j\x1f��S\x1b!�~\x1b\x14\x01�HH��\x14�S\x13�q5Npg�dRW�awu\x13\x05�S��$�d�Q\x14�ps>�\x0e\x17�nY\x1d��\x07���@��|X\r�<�\x08+\'KO��\x0e1H\x0b\x0bV8���h\x07�J\

In [22]:
data = json.loads(response_text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)